# [Part1] 마켓컬리 상품리뷰 크롤링

## 설명
- 1. '인기상품순' 이동
- 2. 상품명, 부연설명, 가격, 해당 URL 링크 크롤링
- 3. 후기 크롤링 1000개씩 (for) // 1페이지는 공지사항있음, 2페이지부터 // 700 개 * 300
- 4. 상품명 별 크롤링 1000개씩 데이터프레임 저장
- 5. csv로 저장(상품명, 부연설명, 가격, 해당 URL, 상품리뷰)

# 1. 환경 설정

In [1]:
from selenium import webdriver 
from selenium.webdriver import ActionChains 
import time
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

driver = webdriver.Chrome() 
driver.set_window_size(1500, 800)

# 1. 사이트 접속
driver.get('https://www.kurly.com/shop/goods/goods_list.php?category=029')
time.sleep(4) #로딩 대기

# 2. '인기상품순' 클릭 
driver.find_element_by_css_selector('#goodsList > div.list_ability > div > div > div > ul > li:nth-child(3) > a').click()

# 2. 인기상품군(280여건) 의 상품정보 크롤링 

#### 2.1) 상품명, 상품가격, 상품설명, 제품코드 함수생성

In [2]:
## 1. 상품명, 상품가격, 상품설명 크롤링
def get_product_info():
    product_info = driver.find_elements(By.CLASS_NAME, "info")
    for product_list in product_info:

        # 할인 가격 시 밀리는 현상 발생
        if ("%" in product_list.text.split('\n')[1]):
            ## 상품명
            product_name = product_list.text.split('\n')[0]
            product_dict['product_name'].append(product_name)

            ## 상품가격
            product_price = product_list.text.split('\n')[2]
            product_dict['product_price'].append(product_price)

            ## 상품설명이 없는 경우도 있음.
            try:
                product_content = product_list.text.split('\n')[3]
                product_dict['product_content'].append(product_content)
            except:
                product_content = 'NA'
                product_dict['product_content'].append(product_content)
                pass

        else:            
            # 상품명
            product_name = product_list.text.split('\n')[0]
            product_dict['product_name'].append(product_name)

            # 상품가격
            product_price = product_list.text.split('\n')[1]
            product_dict['product_price'].append(product_price)

            # 상품설명
            product_content = product_list.text.split('\n')[2]
            product_dict['product_content'].append(product_content)
            
        
## 2. 해당 제품코드 크롤링
def get_product_id():
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    contents = soup.select('div.group_btn')

    for p_tag in contents:
        try:
            # 제품코드
            product_list.append(int(p_tag.text))
        except:
            pass

#### 2.2) 상품 정보 크롤링 실행

In [3]:
# 설정
product_list = []
product_dict = {'product_name':[], 'product_price' : [], 'product_content' : []}


# 1번째 페이지 실행 
get_product_id()
get_product_info()
    
# 2번쨰 페이지 클릭
driver.find_element_by_css_selector('#goodsList > div.layout-pagination > div > span:nth-child(4) > a').click()
time.sleep(2)
get_product_id()
get_product_info()

# 3번째 페이지 클릭
driver.find_element_by_css_selector('#goodsList > div.layout-pagination > div > span:nth-child(5) > a').click()
time.sleep(2)
get_product_id()
get_product_info()
print("제품코드 갯수 : ", len(product_list))

# Product_id + URL 작업
review_url = []
for product_url in product_list:
    concat_url = "https://www.kurly.com/shop/goods/goods_review_list.php?goodsno=" + str(product_url) + "&page="
    review_url.append(concat_url)

# 품절된 상품은 크롤링 되지 않으므로, 상태에 맞춰 변경 필요
prodcut_df = pd.DataFrame.from_dict(product_dict)[:-8] # 8개의 상품 품절 상태
prodcut_df['product_code'] = product_list
prodcut_df['product_url'] = review_url

279
279


# 3. 인기상품군의 상품리뷰 크롤링 

#### 3.1) 해당 상품평 크롤링 과정

In [10]:
# 설정
review_url = prodcut_df['product_url'].to_list()[:3] # 3개의 상품만
# 한 리뷰 당 몇개의 게시글을 크롤링, 임시로 4페이지까지, 한 페이지당 7개의 리뷰 게시글 확인 
page_limit = 10 # 50 개하면 350개리뷰 크롤링
kurly_dict = {'review':[]}
review_list = []


# 1. 280 개 중 1개의 상품주소로 접속
for URL in review_url:
    
    # 2. 해당 상품주소 중 n 개의 리뷰 크롤링
    for page_num in range(2, page_limit+1):
        if (page_num-1) % 10 == 0:
            print('{}번째 페이지 수집 중'.format(seq), time.time() - start, sep = '\t')

        # 페이지에서 게시물 리스트가 포함된 프레임으로 이동
        driver.get(URL + str(page_num))
    
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        contents = soup.select('div.inner_review')

        for p_tag in contents:
            review = str(p_tag).split("</div>")[-2].strip()
            kurly_dict['review'].append(review)
        time.sleep(int(np.random.randint(2, 5, size=1))) # 2 ~ 5초 간 랜덤으로 sleep

# 3. 데이터프레임 변경
Final_df = pd.DataFrame.from_dict(kurly_dict)

# 4. 중복된 리뷰 제거
Final_df.drop_duplicates(inplace = True)
Final_df

,review
0,생각보다 별로인 것
1,맛잇는데요 ㅎㅎㅎㅎ
2,매워요 ㅎㅎㅎㅎㅎㅎㅎ
3,랜덤으로 많이 들은 날도 있고 아닌 날도 있고 ...그르네요 ... 맛있긴 해요
4,요 또 주문 ㅎㅎㅎ
...,...
184,살짝 짜긴 한데 그래도 나트륨양이 너무 높진 않아 괜찮은것 같아요
185,살짝 짜긴 한데 그래도 나트륨양이 너무 높진 않아 괜찮은것 같아요
186,살짝 짜긴 한데 그래도 나트륨양이 너무 높진 않아 괜찮은것 같아요
187,살짝 짜긴 한데 그래도 나트륨양이 너무 높진 않아 괜찮은것 같아요


- 1) 어떤 상품에 대한 리뷰는 아직 안넣음 -> 넣으면 데이터 용량 커지고.. 분리하는 게 나을듯, 어떤 상품에 대한 리뷰까지 봐야할까?
- 2) 그리고 몇개의 리뷰를 넣을지 고민 350개리뷰 * 280개의 상품 = 10만개

# 4. 저장

In [ ]:
# 상품정보
prodcut_df.to_csv('product_df.csv', index = False, encoding = 'utf-8-sig')

# 상품리뷰
Final_df.to_csv('product_df.csv', index = False, encoding = 'utf-8-sig')